# Trees

This notebook describes Stacks, Queues and Deques in Python. It covers the following:

1. Trees
2. Binary Trees

For each concept, there are Python examples which help illustrate the ideas.

In [ ]:
class Tree:
    """Abstract base class representing a tree structure."""

    #------------------------------- nested Position class -------------------------------
    class Position:
        """An abstraction representing the location of a single element."""
        def element(self):
            """Return the element stored at this Position."""
            raise NotImplementedError("must be implemented by subclass")

        def __eq__(self, other):
            """Return True if other Position represents the same location."""
            raise NotImplementedError("must be implemented by subclass")

        def __ne__(self, other):
            """Return True if other does not represent the same location."""
            return not (self == other) # opposite of eq

    # ---------- abstract methods that concrete subclass must support ----------
    def root(self):
        """Return Position representing the tree s root (or None if empty)."""
        raise NotImplementedError("must be implemented by subclass")

    def parent(self, p):
        """Return Position representing p s parent (or None if p is root)."""
        raise NotImplementedError("must be implemented by subclass")

    def num_children(self, p):
        """Return the number of children that Position p has."""
        raise NotImplementedError("must be implemented by subclass")

    def children(self, p):
        """Generate an iteration of Positions representing p s children."""
        raise NotImplementedError("must be implemented by subclass")

    def __len__(self):
        """Return the total number of elements in the tree."""
        raise NotImplementedError("must be implemented by subclass")
        
    def positions(self):
        """Generate an iteration of the tree s positions"""
        raise NotImplementedError("must be implemented by subclass")

    # ---------- concrete methods implemented in this class ----------
    def is_root(self, p):
        """Return True if Position p represents the root of the tree."""
        return self.root() == p

    def is_leaf(self, p):
        """Return True if Position p does not have any children."""
        return self.num_children(p) == 0

    def is_empty(self):
        """Return True if the tree is empty."""
        return len(self) == 0
    
    def __iter__(self):
        """Generate an iteration of the tree's elements."""
        for p in self.positions(): # use same order as positions()
            yield p.element()      # but yield each element
    
    def depth(self, p):
        """
        Return the number of levels separating Position p from the root.
        This runs in O(n) worse case
        """
        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p))
    
    def _height1(self):
        """
        Return the height of the tree. 
        This implementation runs in O(n^2) in the worst case
        """
        return max(self.depth(p) for p in self.positions() if self.is_leaf(p)) # note: need to define positions
    
    def _height2(self, p):
        """
        Return the height of the tree. 
        This implementation runs in O(n)
        """
        if self.is_leaf(p):
            return 0
        else:
            return 1 + max([_height2(c) for c in self.children(p)])
        
    def height(self, p=None):
        """
        Return the height of the subtree rooted at Position p.
        If p is None, return the height of the entire tree.
        This is the public interface of height 
        (it calls _height to compute the height of the tree)
        """
        if p is None:
            p = self.root()
        return self._height2(p) # start height2 recursion

Let $d_p$ be the depth of node $p$, $c_p$ be the number of children of node $p$, and $n$ be the number of nodes in the tree

For our algorithmic analysis of `_height1` and `_height2`, we assume that:
* `self.positions()` runs in $O(n)$
* `self.children(p)` runs in $O(c_p + 1)$

We can see that `self.depth(p)` runs in $O(d_p + 1)$, since the recursion is called once for $p$ and once for each ancestor of $p$.
In the worst case this ends up being $O(n)$ worst case if the tree has one branch.

We compare two ways of computing the height of the tree:

1) `_height1` runs in $O(n^2)$ in the worst case. This is since:
* `self.positions()` gets called once +
* `self.depth(p)` gets called for each $p \in L$ leaf nodes
* So this runs in time $O(n + \sum_{p \in L} (d_p + 1))$
* this is $O(n^2)$ in the worst case (see below)

2) calling `_height2` on the root node of the tree. This runs in $O(n)$ in the worst case.
* Each non-recursive call of `_height2` invokes `self.children` (plus `max`, `+`)
* There are $n$ calls of `_height2`, so $O(\sum_p (c_p+1))=O(n+\sum_p c_p)$
* We note that $\sum_p c_p = n-1$, since every node (minus the root) has *one* parent
* Hence, $O(n+\sum_p c_p) = O(n + n - 1) = O(n)$

To show that `_height1` runs in $O(n^2)$ worst case. We find a tree s.t. $D=\sum_{p\in L} d_p = \Omega(n^2)$ (which is excercize C.33 of Chp 8 )

The tree is constructed by:
* a chain of $\frac{n}{2}$ nodes
* The leaf node of the chain is the root of a complete binary subtree containing the remaining $\frac{n}{2}$ nodes
* Note that: $\sum_{p\in L} d_p = \sum_{p\in L} \frac{n}{2} + \log(\frac{n}{2})$ 
* Since there are $\approx 2^{\log\frac{n}{2}}$ leaf nodes, $=\frac{n}{2} + \log(\frac{n}{2})$

## 2. Binary Trees

In [ ]:
class BinaryTree(Tree):
    """Abstract base class representing a binary tree structure."""

    # --------------------- additional abstract methods ---------------------
    def left(self, p):
        """
        Return a Position representing p's left child.
        Return None if p does not have a left child.
        """
        raise NotImplementedError('must be implemented by subclass')

    def right(self, p):
        """
        Return a Position representing p's right child.
        Return None if p does not have a right child.
        """
        raise NotImplementedError('must be implemented by subclass')

    # ---------- concrete methods implemented in this class ----------
    def sibling(self, p):
        """Return a Position representing p's sibling (or None if no sibling)."""
        parent = self.parent(p)
        if parent is None:
            return None # root has no siblings
        if p == self.left(parent):
            return self.right(parent)
        else:
            return self.left(parent)

    def children(self, p):
        """Generate an iteration of Positions representing p's children."""
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)

In [ ]:
from collections import deque

class LinkedBinaryTree(BinaryTree):
    """
    Linked representation of a binary tree structure.
    Note: this has O(n) space complexity
          the run time of all methods are O(1) except for computing depth and height
    """

    class _Node: # Lightweight, nonpublic class for storing a node.

        __slots__ = '_element' , '_parent' , '_left' , '_right', '_x', '_y'

        def __init__(self, element, parent=None, left=None, right=None):
            self._element = element
            self._parent = parent
            self._left = left        # left child
            self._right = right      # right child
            self._x = None           # stores x spacial info for printout
            self._y = None           # stores y spacial info for printout

        def __repr__(self):
            return str(self._element)

    class Position(BinaryTree.Position):
        """An abstraction representing the location of a single element."""
        def __init__(self, container, node):
            """Constructor should not be invoked by user."""
            self._container = container
            self._node = node

        def element(self):
            """Return the element stored at this Position."""
            return self._node._element

        def __eq__(self, other):
            """Return True if other is a Position representing the same location."""
            return type(other) is type(self) and other._node is self._node

        def __repr__(self):
            return str(self._node)

    def _validate(self, p):
        """Return associated node, if position is valid."""
        if not isinstance(p, self.Position):
            raise TypeError("p must be proper Position type")
        if p._container is not self:
            raise ValueError("p does not belong to this container")
        if p._node._parent is p._node: # convention for deprecated nodes
            raise ValueError("p is no longer valid")
        return p._node

    def _make_position(self, node):
        """Return Position instance for given node (or None if no node)."""
        return self.Position(self, node) if node is not None else None

    #-------------------------- binary tree constructor --------------------------
    def __init__(self):
        """Create an initially empty binary tree."""
        self._root = None
        self._size = 0

    #-------------------------- public accessors --------------------------
    def __len__(self):
        """Return the total number of elements in the tree."""
        return self._size

    def root(self):
        """Return the root Position of the tree (or None if tree is empty)."""
        return self._make_position(self._root)

    def parent(self, p):
        """Return the Position of p's parent (or None if p is root)."""
        node = self._validate(p)
        return self._make_position(node._parent)

    def left(self, p):
        """Return the Position of p's left child (or None if no left child)."""
        node = self._validate(p)
        return self._make_position(node._left)

    def right(self, p):
        """Return the Position of p s right child (or None if no right child)."""
        node = self._validate(p)
        return self._make_position(node._right)

    def num_children(self, p):
        """Return the number of children of Position p"""
        node = self._validate(p)
        if node is not None:
            return len([c for c in self.children(p)])
        return 0

    def _add_root(self, e):
        """
        Place element e at the root of an empty tree and return new Position.
        Raise ValueError if tree nonempty.
        """
        if self._root is not None:
            raise ValueError('Root exists')
        self._size = 1
        self._root = self._Node(e)
        return self._make_position(self._root)

    def _add_left(self, p, e):
        """
        Create a new left child for Position p, storing element e.
        Return the Position of new node.
        Raise ValueError if Position p is invalid or p already has a left child.
        """
        node = self._validate(p)
        if node._left is not None: 
            raise ValueError("Left child exists")
        node._left = self._Node(e, node)
        self._size += 1
        return self._make_position(node._left)

    def _add_right(self, p, e):
        """
        Create a new right child for Position p, storing element e.
        Return the Position of new node.
        Raise ValueError if Position p is invalid or p already has a right child.
        """
        node = self._validate(p)
        if node._right is not None: 
            raise ValueError("Right child exists")
        node._right = self._Node(e, node)
        self._size += 1
        return self._make_position(node._right)
    
    def _replace(self, p, e):
        """Replace the element at position p with e, and return old element."""
        node = self._validate(p)
        old = node._element
        node._element = e
        return old
    
    def _delete(self, p):
        """
        Delete the node at Position p, and replace it with its child, if any.
        Return the element that had been stored at Position p.
        Raise ValueError if Position p is invalid or p has two children.
        """
        node = self._validate(p)

        if node.num_children() == 2:
            raise ValueError('Node must have exactly one child')

        child = node._left if node._left else node._right # could be None if no children

        if child is not None:
            child._parent = node._parent
        
        if node is self._root:
            self._root = child # child becomes root
        else:
            #  set grandparent to be parent of child
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child

        self._size -= 1
        node._parent = node # convention for deprecated node
        return node._element

    def _attach(self, p, t1, t2):
        """Attach trees t1 and t2 as left and right subtrees of external p."""
        node = self._validate(p)

        if not self.is_leaf(p):
            raise ValueError('position must be leaf') 

        if not type(self) is type(t1) is type(t2): # all 3 trees must be same type
            raise TypeError('Tree types must match')

        self._size += len(t1) + len(t2)
        
        if not t1.is_empty():
            node._left = t1._root
            t1._root._parent = node
            t1._root = None
            t1._size = 0

        if not t2.is_empty():
            node._left = t2._root
            t2._root._parent = node
            t2._root = None
            t2._size = 0
    
    #-------------------------- traversals --------------------------
    #  Should include these in base Tree class!
    
    def positions(self, how='preorder'):
        """Generate an iteration of the tree's positions"""
        if how == 'preorder':
            return self.preorder()
        elif how == 'postorder':
            return self.postorder()
        elif how == 'breadthfirst':
            return self.breadthfirst()
        elif how == 'inorder':
            return self.inorder()
    
    def preorder(self):
        """Generate a preorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()): # start recursion
                yield p
    
    def _subtree_preorder(self, p):
        """Generate a preorder iteration of positions in subtree rooted at p."""
        yield p                                     # visit p before its subtrees
        for c in self.children(p):                  # for each child c
            for other in self._subtree_preorder(c): # do preorder of c’s subtree
                yield other                         # (re)yielding each in outer context
    
    def postorder(self):
        """Generate a postorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield p

    def _subtree_postorder(self, p):
        """Generate a postorder iteration of positions in subtree rooted at p."""
        for c in self.children(p):                   # for each child c
            for other in self._subtree_postorder(c): # do postorder of c’s subtree
                yield other                          # (re)yielding each in outer context
        yield p                                      # visit p after its subtrees

    def breadthfirst(self):
        """Generate a breadth-first iteration of the positions of the tree."""
        if not self.is_empty():
            D = deque() # treat like queue: enqueue = D.append (add to right)
                        #                   dequeue = D.popleft() (remove from left)

            D.append(self.root())           # Initialize queue Q to contain T.root()

            while len(D) != 0:
                p = D.popleft()             # p is the oldest entry in the queue
                yield p
                for c in self.children(p):  # perform the “visit” action for position p
                    D.append(c)             # add p’s children to the end of the queue for later visits

    def inorder(self):
        if not self.is_empty():
            for e in self._subtree_inorder(self.root()):
                yield e

    def _subtree_inorder(self, p):
        if self.left(p) is not None:
            for e in self._subtree_inorder(self.left(p)):
                yield e
        yield p
        if self.right(p) is not None:
            for e in self._subtree_inorder(self.right(p)):
                yield e

In [ ]:
#  Same tree as in notes
LB = LinkedBinaryTree()
pos0 = LB._add_root('/')
pos1 = LB._add_left(pos0, '*')
pos2 = LB._add_right(pos0, '+')
pos3 = LB._add_left(pos1, '+')
pos4 = LB._add_right(pos1, '4')
pos5 = LB._add_left(pos2, '-')
pos6 = LB._add_right(pos2, '2')
pos7 = LB._add_left(pos3, '3')
pos8 = LB._add_right(pos3, '1')
pos11 = LB._add_left(pos5, '9')
pos12 = LB._add_right(pos5, '5')

In [ ]:
for p in LB.inorder():
    print(p, end=', ')

In [ ]:
for p in LB.preorder():
    print(p, end=', ')

In [ ]:
for p in LB.postorder():
    print(p, end=', ')

In [ ]:
for p in LB.breadthfirst():
    print(p, end=', ')

In [ ]:
class EulerTour:
    """
    Abstract base class for performing Euler tour of a tree.
    hook previsit and hook postvisit may be overridden by subclasses.
    """
    def __init__(self, tree):
        """Prepare an Euler tour template for given tree."""
        self._tree = tree
    
    def tree(self):
        """Return reference to the tree being traversed."""
        return self._tree

    def execute(self):
        """Perform the tour and return any result from post visit of root."""
        if len(self._tree) > 0:
            return self._tour(self._tree.root(), 0, [ ]) # start the recursion

    def _tour(self, p, d, path):
        """
        Perform tour of subtree rooted at Position p.
        p Position of current node being visited
        d depth of p in the tree
        path list of indices of children on path from root to p
        """
        self._hook_previsit(p, d, path) # ”pre visit” p
        results = []
        path.append(0) # add new index to end of path before recursion
        for c in self._tree.children(p):
            results.append(self._tour(c, d+1, path)) # recur on child s subtree
            path[-1] += 1 # increment index
        path.pop() # remove extraneous index from end of path
        answer = self._hook_postvisit(p, d, path, results) # ”post visit” p
        return answer

    def _hook_previsit(self, p, d, path): # can be overridden
        """
        This function is called once for each position, immediately before its subtrees
        (if any) are traversed.
        p is a position in the tree
        d is the depth of that position
        path is a list of indices, using the convention described in
        the discussion of Code Fragment 8.24.
        No return value is expected from this
        function.
        """
        pass

    def _hook_postvisit(self, p, d, path, results): # can be overridden
        """
        This function is called once for each position, immediately after its subtrees
        (if any) are traversed. 
        The first three parameters use the same convention as did hook previsit. 
        results is a list of objects that were provided as return values from the post visits of the respective subtrees of p. 
        Any value returned by this call will be available to the parent of p during its postvisit.
        """
        pass

In [ ]:
class BinaryLayout(EulerTour):
    """
    Class for computing (x,y) coordinates for each node of a binary tree.
    TODO: fix these!
    """
    def __init__(self, tree):
        super().__init__(tree) # must call the parent constructor
        self._count = 0 # initialize count of processed nodes

    def _hook_postvisit(self, p, d, path, results):
        p._node._x = self._count # x-coordinate serialized by count
        p._node._y = d           # y-coordinate is depth
        self._count += 1         # advance count of processed nodes


In [ ]:
BL = BinaryLayout(LB)
out = BL.execute()

for p in LB.positions():
    print(p._node, p._node._x, p._node._y)

In [ ]:
#. Not good so far!
#import matplotlib.pyplot as plt

#for p in LB.positions():
#    plt.text(p._node._x, p._node._y, str(p._node), size=50)